### Predict items to buy

Need to use (user, sku) pairs for the prediction

Use simple analysis to look at the probability


In [1]:
import base
import numpy as np
import pandas as pd

pus = base.ProductUserSet()

In [22]:
data = pd.DataFrame(pus.get_user_sku_action('favor', pus.TYPE_FAVOR, 0, 6e6))
data = data.join(pus.get_user_sku_action('buy1', pus.TYPE_BUY, 0, 3e6), how='outer')
data = data.join(pus.get_user_sku_action('buy2', pus.TYPE_BUY, 3e6, 5e6), how='outer')
data = data.join(pus.get_user_sku_action('buy3', pus.TYPE_BUY, 5e6, 6e6), how='outer')
data = data.join(pus.get_user_sku_action('view1', pus.TYPE_VIEW, 0, 3e6), how='outer')
data = data.join(pus.get_user_sku_action('view2', pus.TYPE_VIEW, 3e6, 5e6), how='outer')
data = data.join(pus.get_user_sku_action('view3', pus.TYPE_VIEW, 5e6, 6e6), how='outer')
data = data.join(pus.get_user_sku_cart('cart1', 0, 3e6, True), how='outer')
data = data.join(pus.get_user_sku_cart('cart2', 0, 6e6, True), how='outer')
data = data.join(pus.get_user_sku_action('click1', pus.TYPE_CLICK, 0, 3e6), how='outer')
data = data.join(pus.get_user_sku_action('click2', pus.TYPE_CLICK, 3e6, 5e6), how='outer')
data = data.join(pus.get_user_sku_action('click3', pus.TYPE_CLICK, 5e6, 6e6), how='outer')
data = data.join(pus.get_user_sku_action('final', pus.TYPE_BUY, 6e6, 6e6 + 5 * 24 *3600), how='outer')
data[pd.isnull(data)] = 0

In [33]:
def factor_analyse(col1, col2, dt=data, print_out=True):
    f1 = dt[col1] > 0
    f2 = dt[col2] > 0
    num1 = np.sum(np.logical_and(f1, f2))
    num2 = np.sum(np.logical_and(f1, np.logical_not(f2)))
    num3 = np.sum(np.logical_and(np.logical_not(f1), f2))
    num4 = np.sum(np.logical_not(np.logical_or(f1, f2)))
    if print_out:
        print '\t{}\t{}'.format(col2, '-')
        print '{}\t{}\t{}'.format(col1, num1, num2)
        print '{}\t{}\t{}'.format('-', num3, num4)
    return num1, num2, num3, num4

In [24]:
buy_stat = pd.DataFrame()
for col in data.columns:
    buy_stat[col] = factor_analyse(col, 'final')

import pickle
with open('data/probability', 'w') as f:
    pickle.dump(buy_stat, f)

	final	-
favor	404	90337
-	4381	3465433
	final	-
buy1	29	15068
-	4756	3540702
	final	-
buy2	46	18537
-	4739	3537233
	final	-
buy3	45	7945
-	4740	3547825
	final	-
view1	653	1390334
-	4132	2165436
	final	-
view2	1272	1601556
-	3513	1954214
	final	-
view3	2163	758356
-	2622	2797414
	final	-
cart1	169	89571
-	4616	3466199
	final	-
cart2	1213	210650
-	3572	3345120
	final	-
click1	629	1356074
-	4156	2199696
	final	-
click2	1227	1533349
-	3558	2022421
	final	-
click3	2089	734729
-	2696	2821041
	final	-
final	4785	0
-	0	3555770


In [29]:
for col in buy_stat.columns:
    prob1 = buy_stat[col][0] * 1.0 / (buy_stat[col][0] + buy_stat[col][1])
    prob2 = buy_stat[col][2] * 1.0 / (buy_stat[col][2] + buy_stat[col][3])
    if prob2 == 0.0:
        prob2 += 1e-8
    print col, '\t+\t', prob1, '\t-\t', prob2, '\tratio\t', prob1 / prob2

favor 	+	0.0044522321773 	-	0.00126260370152 	ratio	3.5262308925
buy1 	+	0.00192091143936 	-	0.0013414345904 	ratio	1.43198293313
buy2 	+	0.00247538072432 	-	0.00133795524075 	ratio	1.85012222302
buy3 	+	0.00563204005006 	-	0.001334247227 	ratio	4.22113678491
view1 	+	0.0004694508288 	-	0.00190452661544 	ratio	0.246492133528
view2 	+	0.000793597316743 	-	0.00179442792585 	ratio	0.442256445805
view3 	+	0.00284411040355 	-	0.000936416531787 	ratio	3.03722788632
cart1 	+	0.00188321818587 	-	0.00132994700092 	ratio	1.41600994969
cart2 	+	0.00572539801664 	-	0.00106668514154 	ratio	5.36746767501
click1 	+	0.000463623947172 	-	0.00188578906388 	ratio	0.245851434847
click2 	+	0.000799569392458 	-	0.00175618799603 	ratio	0.455286902238
click3 	+	0.00283516417894 	-	0.000954763138352 	ratio	2.96949480458
final 	+	1.0 	-	1e-08 	ratio	100000000.0


In [38]:
# More complex interactions
for col in data.columns:
    if col != 'final':
        sub_set = data[data[col] > 0]
        print col, "+ subset"

        for col2 in sub_set.columns:
            if col2 != col and col2 != 'final':
                ret = factor_analyse(col2, 'final', sub_set, False)
                prob1 = ret[0] * 1.0 / (ret[0] + ret[1])
                prob2 = ret[2] * 1.0 / (ret[2] + ret[3])
                if ret[2] == 0:
                    prob2 = 1.0 / (ret[2] + ret[3])
                print col2, '\t', prob1, '\t', prob2, '\t', prob1 / prob2
    print

favor + subset
buy1 	0.000714796283059 	0.00451075641915 	0.158464837521
buy2 	0.000908677873694 	0.00454032075898 	0.200135171485
buy3 	0.00312174817898 	0.00446647360214 	0.698929056132
view1 	0.00279541943552 	0.00580464371497 	0.481583293099
view2 	0.00481237467774 	0.00396080883886 	1.21499796469
view3 	0.0121173469388 	0.00070550788364 	17.17535299
cart1 	0.00365390236773 	0.00451733632105 	0.808862149736
cart2 	0.0104371682472 	0.00309568226674 	3.37152438391
click1 	0.00272465941757 	0.00581441185746 	0.468604475288
click2 	0.00487270307822 	0.00391019172553 	1.24615451626
click3 	0.0121338483098 	0.000796955305445 	15.2252557037

buy1 + subset
favor 	0.000714796283059 	0.00204409402833 	0.349688553048
buy2 	0.0144927536232 	0.00174613834788 	8.29988851728
buy3 	0.03125 	0.00179604869288 	17.3993055556
view1 	0.00190826688475 	0.00235849056604 	0.809105159136
view2 	0.00254108080637 	0.00152273221666 	1.6687640667
view3 	0.00807217473884 	0.000923716419059 	8.73880183602
cart1 

#### Summary

Favor, previous purchase, adding to cart will increase probability for buying.

Recent click/viewing increases probability.


But if an item has been favored, it will for sure not be bought again.

If an item has been bought many times, it will be bought again.


In [40]:
train_data = data.drop('final', 1).as_matrix()
train_label = np.array(data['final'] > 0, dtype=int)
mean_dt = np.mean(train_data, axis=0)
std_dt = np.std(train_data, axis=0)
train_data = (train_data - mean_dt) / std_dt

In [42]:
nrow = len(train_data)
test_size = int(nrow * 0.3)
import random
idx = random.sample(xrange(nrow), test_size)
mask = np.ones(nrow, dtype=bool)
mask[idx] = False
np.save('train_data2', train_data[mask, :])
np.save('train_label2', train_label[mask])
np.save('test_data2', train_data[idx, :])
np.save('test_label2', train_label[idx])